# Agent-based modeling

---
## Before Class
1. Install ffmpeg: `$ conda install ffmpeg`
* Review game of life


---
## Learning Objectives
1. Implement agent-based modeling
* Implement animation in python


---
## Agent-based modeling

Today we will be implementing a version of agent-based modeling or a cellular automaton. A classic example of this is Conway's Game of Life in which there are many 'cells' in a grid and their fate is determined by a set of simple rules:

1. Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by overpopulation.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The fate of the cells is entirely dependant on the initial state of the system.

<img src="figures/gol.gif">



In [1]:
# You will need these imports
import numpy as np 
import matplotlib.pyplot as plt  
from matplotlib import animation, rc 
from IPython.display import HTML

In [2]:
# Define some global variables
DEAD = 0
ALIVE = 1

In [3]:
def random_grid(dim, fraction_alive, seed): 
    """ Initialize a random grid of dim x dim random values
    
    Args:
        dim (int): dimensions for grid (will be square so x = y = dim)
        fraction_alive (float): initial fraction of live cells on grid
        seed (int): seed for np.random.choice
    
    Returns:
        grid (numpy matrix): dim x dim numpy matrix populated with live and dead cells
        
    Example:
    >>> random_grid(2, .5, 1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    array([[1, 0], [1, 1]])
    """
    
    np.random.seed(seed)
    #Grid should be square of dim x dim
    values = [ALIVE, DEAD] # Alive is 1, dead is 0
    return np.random.choice(values, dim*dim, p=[fraction_alive, 1-fraction_alive]).reshape(dim, dim) 


In [4]:
def count_living_neighbors(row, column, grid):
    """ Count how many living neighbors exist
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        living_count (int): number of living neighbors
    
    Example:
    >>> count_living_neighbors(2,2, random_grid(2,.5,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    1
    """
    neighbors = get_neighbors(row, column, grid)
    
    count = 0
    for neighbor in neighbors:
        if neighbor == ALIVE:
            count += 1
            
    return count

In [5]:
def get_neighbors(row, column, grid):
    """ Get the neighboring cells
    excluding any that might be out of bounds
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        neighbors (list): list of adjacent cells
        
    Example:
    >>> get_neighbors(2,2, random_grid(2,.5,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    [1]
    """
    
    neighbors = []
    for r in range(row-1, row+2):
        if r < 0:
            continue
        elif r >= grid.shape[0]:
            continue
        
        for c in range(column-1, column+2):
            if c < 0:
                continue
            elif c >= grid.shape[1]:
                continue
            elif r == row and c == column:
                continue
            else:
                neighbors.append(grid[r][c])

    return neighbors

In [6]:
def update_grid(frameNum, img, grid, dim): 
    """ Update grid of cells based on rules:
    
    Any live cell with fewer than two live neighbours dies, as if by underpopulation.
    Any live cell with two or three live neighbours lives on to the next generation.
    Any live cell with more than three live neighbours dies, as if by overpopulation.
    Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

    Args:
        frameNum (int): frame number for the simulation (this is passed by animation)
        img (image object): this is passed by animation
        grid (np array): this is passed by animation
        dim (int): dimensions of grid (this is passed by animation)
        
    This function copies a new Grid over the existing grid for the animation process
    
    """
    newGrid = grid.copy()
    for i in range(dim): 
        for j in range(dim): 
            living_neighbors = count_living_neighbors(i, j, grid)
            #print(living_neighbors)
        
            # apply Conway's rules 
            if grid[i][j] == DEAD: 
                if living_neighbors == 3: 
                    newGrid[i][j] = ALIVE
            else: # Alive
                if living_neighbors != 3 and living_neighbors != 2: 
                    newGrid[i][j] = DEAD
  
    # update data 
    img.set_data(newGrid) 
    grid[:] = newGrid[:] 


In [7]:
def run_simulation(grid_size = 100, fraction_alive = .5, updateInterval = 500, seed=42): 
    """ Function to run the full simulation
    Each frame is an iteration of the model which calls update_grid
    with the arguments listed in fargs
    
    img should be updated with img.set_data(newGrid) in each iteration of update_grid
    
    """
    # declare grid 
    grid = np.array([]) 
    grid = random_grid(grid_size, fraction_alive, seed=seed) 
  
    # set up animation 
    fig, ax = plt.subplots(figsize=(6,6)) 
    img = ax.imshow(grid) 
    anim = animation.FuncAnimation(fig, update_grid, fargs=(img, grid, grid_size, ), 
                                  frames = 100, 
                                  interval=updateInterval) 
    
    #anim.save('gol.gif', writer='imagemagick', fps=60)
    return HTML(anim.to_html5_video())


In [8]:
%%capture
game_of_life = run_simulation()

In [9]:
game_of_life

In [10]:
!ffmpeg

ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 12.0.1
  configuration: --prefix=/Users/alauth/opt/anaconda3/envs/b529 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1645955481026/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Hyper fast Audio and Video encoder
usage: ffmpeg [op